In [ ]:
#이미지 - 랜드마크 - z_value - 유클리드 거리:
#이미지 랜마//유클리드//제트
from google.colab import drive
drive.mount('/content/drive')
!pip install imblearn
# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard
# 감정 레이블 정의
import numpy as np
import random
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score, roc_curve, auc
import pandas as pd


# 시드 설정

In [ ]:
# 시드 설정
import tensorflow as tf
import numpy as np
import random

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

epochs = 100
batch_size =32
patience_es = 10
factor_lr = 0.5
patience_lr = 5
learning_rate_combined = 0.0001

# Image Only

In [ ]:
#이미지 - 랜드마크 - z_value - 유클리드 거리:
#이미지 랜마//유클리드//제트
# from google.colab import drive
# drive.mount('/content/drive')


# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard
# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}
model_name = 'Image Only'


# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [128, 128])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)


    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    return image, label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


'''수정함'''
train_tfrecord_file = "/content/drive/MyDrive/0902_128X128/clean_YES_Z_train_shuffled.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/0902_128X128/clean_YES_Z_val_shuffled.tfrecord"
test_tfrecord_file = '/content/drive/MyDrive/0902_128X128/clean_YES_Z_test_shuffled.tfrecord'


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)


# 모델 정의
image_shape = (128, 128, 1)  # 이미지 데이터의 형태

model_image_input = Input(shape=image_shape)  # 이미지 입력

# 이미지 서브넷
x_image = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', input_shape=(128,128,1))(model_image_input)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.2)(x_image)  # new dropout layer
x_image = MaxPooling2D((2,2))(x_image)
print(f"After first MaxPooling2D: {x_image.shape}")  # 중간 텐서 크기 출력

x_image = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.3)(x_image)  # increased dropout rate
x_image = MaxPooling2D((2,2))(x_image)
print(f"After second MaxPooling2D: {x_image.shape}")  # 중간 텐서 크기 출력

x_image = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.2)(x_image)  # new dropout layer
x_image = MaxPooling2D((2,2))(x_image)
print(f"After third MaxPooling2D: {x_image.shape}")  # 중간 텐서 크기 출력

x_image = Flatten()(x_image)
print(f"After Flatten: {x_image.shape}")  # Flatten 후 텐서 크기 출력

x_image = Dense(64, activation='relu')(x_image)

# 출력 레이어
model_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.001))(x_image)

# 모델 정의
model_combined = Model(inputs= model_image_input, outputs=model_output)


# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)

# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)

import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/0929'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')

plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []


y_test = []

for image, label in test_dataset:
    test_images.append(image)
    y_test.extend(label.numpy())

y_test_pred = model_combined.predict([np.vstack(test_images)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_images)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)



# LANDMARK ONLY

In [ ]:
'''#이미지 - 랜드마크 - z_value - 유클리드 거리:
#이미지 랜마//유클리드//제트
from google.colab import drive
drive.mount('/content/drive')'''

# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}
model_name = 'Landmark Only'
# Define landmark indices
landmark_indices = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46, 300, 293, 334, 296, 336, 285, 295, 282, 283, 276, 33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7, 263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95, 61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 64, 4, 294, 168, 6, 197, 195, 5, 1]
# Define a function to extract selected landmarks



def extract_selected_landmarks(landmarks):
    selected_landmarks = tf.gather(landmarks, indices=landmark_indices, axis=0)
    return tf.reshape(selected_landmarks, [len(landmark_indices) * 3])



# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "landmarks": tf.io.VarLenFeature(tf.float32),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    landmarks = tf.sparse.to_dense(parsed_features["landmarks"])
    landmarks = tf.reshape(landmarks, [-1, 3])
    selected_landmarks = extract_selected_landmarks(landmarks)
    selected_landmarks.set_shape([303])

    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))


    return  selected_landmarks, label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

'''수정함'''
train_tfrecord_file = "/content/drive/MyDrive/0902_128X128/clean_YES_Z_train_shuffled.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/0902_128X128/clean_YES_Z_val_shuffled.tfrecord"
test_tfrecord_file = '/content/drive/MyDrive/0902_128X128/clean_YES_Z_test_shuffled.tfrecord'


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)



# 모델 정의
input_dim = 303  # 랜드마크 데이터의 차원


model_landmark_input = Input(shape=(input_dim,))  # 랜드마크 입력



# 랜드마크 서브넷
# 랜드마크 서브넷
# 랜드마크 서브넷
x_landmark = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_landmark_input)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)  # Dropout rate 조정

x_landmark = Dense(64, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)

x_landmark = Dense(32, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)


num_classes = 7  # 분류할 클래스 수
x_landmark = Dense(num_classes, activation='softmax', kernel_initializer='he_normal')(x_landmark)

# 모델 정의
model_combined = Model(inputs=model_landmark_input, outputs=x_landmark)

# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)


# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)

import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score, auc, precision_recall_curve, average_precision_score

# Assuming you have already defined variables like drive_path, model_name, history, test_dataset, emotion_labels, etc.

accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')

plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)
plt.show()

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_landmarks = []
y_test = []

for landmark, label in test_dataset:
    test_landmarks.append(landmark)
    y_test.extend(label.numpy())

y_test_pred = model_combined.predict([np.vstack(test_landmarks)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)
plt.show()

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)
plt.show()

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_landmarks)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)
plt.show()

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)
plt.show()

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)



# Z_VALUES ONLY

In [ ]:
#오후 3시 8분 수정코드 !! 입력 따로 받기 해놓음
from google.colab import drive
drive.mount('/content/drive')

# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}
model_name = 'ZValue Only'

# Define landmark indices
# Define a function to extract selected landmarks

def calculate_z_value_combinations(z_values):
    z_combinations = list(itertools.combinations(range(8), 3))
    z_product_combinations = []

    for indices in z_combinations:
        selected_values = tf.gather(z_values, indices)
        product = tf.reduce_prod(selected_values)
        z_product_combinations.append(product)

    return tf.stack(z_product_combinations)

# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "label": tf.io.FixedLenFeature([], tf.int64),
        "z_values": tf.io.FixedLenFeature([8], tf.float32),  # 기본값 수정
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    z_values = parsed_features["z_values"]
    z_product_combinations = calculate_z_value_combinations(z_values)

    return z_product_combinations, label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)

# 모델 정의
# z-값 조합의 차원
z_comb_dim = len(list(itertools.combinations(range(8), 3)))

# 1-D CNN 모델 구성
model_z_input = Input(shape=(z_comb_dim,))  # 1차원 데이터 입력 (1차원 배열 형태)

# 완전 연결 레이어
x_z = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_z_input)
x_z = BatchNormalization()(x_z)
x_z = Activation('elu')(x_z)
x_z = Dropout(0.1)(x_z)

model_output = Dense(len(emotion_labels), activation='softmax')(x_z)

# 모델 정의
model_combined = Model(inputs=model_z_input, outputs=model_output)

# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)


# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)


import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')

plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_z_values = []

y_test = []

for  z_value, label in test_dataset:
    y_test.extend(label.numpy())
    test_z_values.append(z_value)

y_test_pred = model_combined.predict([np.vstack(test_z_values)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_z_values)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)

# Image & Landmark


In [ ]:
#오후 3시 8분 수정코드 !! 입력 따로 받기 해놓음


# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2

# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}

model_name = 'Image & Landmark'

# Define landmark indices
landmark_indices = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46, 300, 293, 334, 296, 336, 285, 295, 282, 283, 276, 33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7, 263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95, 61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 64, 4, 294, 168, 6, 197, 195, 5, 1]
# Define a function to extract selected landmarks
def extract_selected_landmarks(landmarks):
    selected_landmarks = tf.gather(landmarks, indices=landmark_indices, axis=0)
    return tf.reshape(selected_landmarks, [len(landmark_indices) * 3])

def calculate_z_value_combinations(z_values):
    z_combinations = list(itertools.combinations(range(8), 3))
    z_product_combinations = []

    for indices in z_combinations:
        selected_values = tf.gather(z_values, indices)
        product = tf.reduce_prod(selected_values)
        z_product_combinations.append(product)

    return tf.stack(z_product_combinations)

# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "landmarks": tf.io.VarLenFeature(tf.float32),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)

    landmarks = tf.sparse.to_dense(parsed_features["landmarks"])
    landmarks = tf.reshape(landmarks, [-1, 3])
    selected_landmarks = extract_selected_landmarks(landmarks)
    selected_landmarks.set_shape([303])

    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    return (image, selected_landmarks), label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)

# z-값 조합의 차원
z_comb_dim = len(list(itertools.combinations(range(8), 3)))

# 모델 정의
input_dim = 303  # 랜드마크 데이터의 차원
image_shape = (48, 48, 1)  # 이미지 데이터의 형태


# 입력 레이어
model_landmark_input = Input(shape=(input_dim,))  # 랜드마크 입력
model_image_input = Input(shape=image_shape)  # 이미지 입력



# 이미지 서브넷




x_image = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', input_shape=(48,48,1))(model_image_input)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.2)(x_image)  # new dropout layer
x_image = MaxPooling2D((2,2))(x_image)
x_image = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.3)(x_image)  # increased dropout rate
x_image = MaxPooling2D((2,2))(x_image)
x_image = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.2)(x_image)  # new dropout layer
x_image = MaxPooling2D((2,2))(x_image)
x_image = Flatten()(x_image)




x_landmark = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_landmark_input)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)  # Dropout rate 조정

x_landmark = Dense(64, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)

x_landmark = Dense(32, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)


# 이미지 서브넷과 랜드마크 서브넷 합치기
combined = Concatenate()([x_image, x_landmark])


combined = Dense(128, kernel_regularizer=l2(0.01))(combined)
combined = BatchNormalization()(combined)
combined = Activation('elu')(combined)
combined = Dropout(0.6)(combined)
# 출력 레이어
model_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(combined)

# 모델 정의
model_combined = Model(inputs=[model_image_input, model_landmark_input], outputs=model_output)

# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)


# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)

import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')

plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []
test_landmarks = []
test_z_values = []

y_test = []

for (image, landmark), label in test_dataset:
    test_images.append(image)
    test_landmarks.append(landmark)
    y_test.extend(label.numpy())

y_test_pred = model_combined.predict([np.vstack(test_images), np.vstack(test_landmarks)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_images), np.vstack(test_landmarks)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)

# Image&Z_value

In [ ]:
#오후 3시 8분 수정코드 !! 입력 따로 받기 해놓음

# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2

# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}
model_name = 'Image & ZValue'

# Define landmark indices
landmark_indices = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46, 300, 293, 334, 296, 336, 285, 295, 282, 283, 276, 33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7, 263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95, 61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 64, 4, 294, 168, 6, 197, 195, 5, 1]
# Define a function to extract selected landmarks
def extract_selected_landmarks(landmarks):
    selected_landmarks = tf.gather(landmarks, indices=landmark_indices, axis=0)
    return tf.reshape(selected_landmarks, [len(landmark_indices) * 3])

def calculate_z_value_combinations(z_values):
    z_combinations = list(itertools.combinations(range(8), 3))
    z_product_combinations = []

    for indices in z_combinations:
        selected_values = tf.gather(z_values, indices)
        product = tf.reduce_prod(selected_values)
        z_product_combinations.append(product)

    return tf.stack(z_product_combinations)

# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "landmarks": tf.io.VarLenFeature(tf.float32),
        "label": tf.io.FixedLenFeature([], tf.int64),
        "z_values": tf.io.FixedLenFeature([8], tf.float32),  # 기본값 수정
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)

    landmarks = tf.sparse.to_dense(parsed_features["landmarks"])
    landmarks = tf.reshape(landmarks, [-1, 3])
    selected_landmarks = extract_selected_landmarks(landmarks)
    selected_landmarks.set_shape([303])

    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    z_values = parsed_features["z_values"]
    z_product_combinations = calculate_z_value_combinations(z_values)

    return (image, z_product_combinations), label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)

# z-값 조합의 차원
z_comb_dim = len(list(itertools.combinations(range(8), 3)))

# 모델 정의
input_dim = 303  # 랜드마크 데이터의 차원
image_shape = (48, 48, 1)  # 이미지 데이터의 형태
z_comb_dim = len(list(itertools.combinations(range(8), 3)))  # z-값 조합의 차원

# 입력 레이어
model_landmark_input = Input(shape=(input_dim,))  # 랜드마크 입력
model_image_input = Input(shape=image_shape)  # 이미지 입력
model_z_input = Input(shape=(z_comb_dim,))  # z-값 조합 입력


# 이미지 서브넷
x_image = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', input_shape=(48,48,1))(model_image_input)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.2)(x_image)  # new dropout layer
x_image = MaxPooling2D((2,2))(x_image)
x_image = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.3)(x_image)  # increased dropout rate
x_image = MaxPooling2D((2,2))(x_image)
x_image = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.2)(x_image)  # new dropout layer
x_image = MaxPooling2D((2,2))(x_image)
x_image = Flatten()(x_image)



x_z = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_z_input)
x_z = BatchNormalization()(x_z)
x_z = Activation('elu')(x_z)
x_z = Dropout(0.1)(x_z)



# 이미지 서브넷, 랜드마크 서브넷과 z_value 서브넷 합치기
combined = Concatenate()([x_image, x_z])

combined = Dense(128, kernel_regularizer=l2(0.01))(combined)
combined = BatchNormalization()(combined)
combined = Activation('elu')(combined)
combined = Dropout(0.6)(combined)
# 출력 레이어
model_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(combined)

# 모델 정의
model_combined = Model(inputs=[model_image_input, model_z_input], outputs=model_output)

# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)


# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)


import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')

plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []
test_landmarks = []
test_z_values = []

y_test = []

for (image, z_value), label in test_dataset:
    test_images.append(image)
    y_test.extend(label.numpy())
    test_z_values.append(z_value)

y_test_pred = model_combined.predict([np.vstack(test_images), np.vstack(test_z_values)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_images), np.vstack(test_z_values)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)


# landmark & z_value

In [ ]:
#오후 3시 8분 수정코드 !! 입력 따로 받기 해놓음


# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2

# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}

model_name = 'Landmark & ZValue'

# Define landmark indices
landmark_indices = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46, 300, 293, 334, 296, 336, 285, 295, 282, 283, 276, 33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7, 263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95, 61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 64, 4, 294, 168, 6, 197, 195, 5, 1]
# Define a function to extract selected landmarks
def extract_selected_landmarks(landmarks):
    selected_landmarks = tf.gather(landmarks, indices=landmark_indices, axis=0)
    return tf.reshape(selected_landmarks, [len(landmark_indices) * 3])

def calculate_z_value_combinations(z_values):
    z_combinations = list(itertools.combinations(range(8), 3))
    z_product_combinations = []

    for indices in z_combinations:
        selected_values = tf.gather(z_values, indices)
        product = tf.reduce_prod(selected_values)
        z_product_combinations.append(product)

    return tf.stack(z_product_combinations)

# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "landmarks": tf.io.VarLenFeature(tf.float32),
        "label": tf.io.FixedLenFeature([], tf.int64),
        "z_values": tf.io.FixedLenFeature([8], tf.float32),  # 기본값 수정
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)

    landmarks = tf.sparse.to_dense(parsed_features["landmarks"])
    landmarks = tf.reshape(landmarks, [-1, 3])
    selected_landmarks = extract_selected_landmarks(landmarks)
    selected_landmarks.set_shape([303])

    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    z_values = parsed_features["z_values"]
    z_product_combinations = calculate_z_value_combinations(z_values)

    return (selected_landmarks, z_product_combinations), label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)

# z-값 조합의 차원
z_comb_dim = len(list(itertools.combinations(range(8), 3)))

# 모델 정의
input_dim = 303  # 랜드마크 데이터의 차원
z_comb_dim = len(list(itertools.combinations(range(8), 3)))  # z-값 조합의 차원

# 입력 레이어
model_landmark_input = Input(shape=(input_dim,))  # 랜드마크 입력
model_z_input = Input(shape=(z_comb_dim,))  # z-값 조합 입력


# 이미지 서브넷


# 랜드마크 서브넷
x_landmark = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_landmark_input)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)  # Dropout rate 조정

x_landmark = Dense(64, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)

x_landmark = Dense(32, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)


# z-값 서브넷
x_z = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_z_input)
x_z = BatchNormalization()(x_z)
x_z = Activation('elu')(x_z)
x_z = Dropout(0.1)(x_z)

# 이미지 서브넷, 랜드마크 서브넷과 z_value 서브넷 합치기
combined = Concatenate()([x_landmark, x_z])

combined = Dense(128, kernel_regularizer=l2(0.01))(combined)
combined = BatchNormalization()(combined)
combined = Activation('elu')(combined)
combined = Dropout(0.6)(combined)
# 출력 레이어
model_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(combined)

# 모델 정의
model_combined = Model(inputs=[model_landmark_input, model_z_input], outputs=model_output)

# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)


# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)
import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')

plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []
test_landmarks = []
test_z_values = []

y_test = []

for (landmark, z_value), label in test_dataset:
    test_landmarks.append(landmark)
    y_test.extend(label.numpy())
    test_z_values.append(z_value)

y_test_pred = model_combined.predict([np.vstack(test_landmarks), np.vstack(test_z_values)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_landmarks), np.vstack(test_z_values)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)

# Image & Landmark & Z_value


In [ ]:
#오후 3시 8분 수정코드 !! 입력 따로 받기 해놓음


# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2

# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}

model_name = 'Image & Landmark & ZValue'

# Define landmark indices
landmark_indices = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46, 300, 293, 334, 296, 336, 285, 295, 282, 283, 276, 33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7, 263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95, 61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 64, 4, 294, 168, 6, 197, 195, 5, 1]
# Define a function to extract selected landmarks
def extract_selected_landmarks(landmarks):
    selected_landmarks = tf.gather(landmarks, indices=landmark_indices, axis=0)
    return tf.reshape(selected_landmarks, [len(landmark_indices) * 3])

def calculate_z_value_combinations(z_values):
    z_combinations = list(itertools.combinations(range(8), 3))
    z_product_combinations = []

    for indices in z_combinations:
        selected_values = tf.gather(z_values, indices)
        product = tf.reduce_prod(selected_values)
        z_product_combinations.append(product)

    return tf.stack(z_product_combinations)

# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "landmarks": tf.io.VarLenFeature(tf.float32),
        "label": tf.io.FixedLenFeature([], tf.int64),
        "z_values": tf.io.FixedLenFeature([8], tf.float32),  # 기본값 수정
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)

    landmarks = tf.sparse.to_dense(parsed_features["landmarks"])
    landmarks = tf.reshape(landmarks, [-1, 3])
    selected_landmarks = extract_selected_landmarks(landmarks)
    selected_landmarks.set_shape([303])

    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    z_values = parsed_features["z_values"]
    z_product_combinations = calculate_z_value_combinations(z_values)

    return (image, selected_landmarks, z_product_combinations), label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)

# z-값 조합의 차원
z_comb_dim = len(list(itertools.combinations(range(8), 3)))

# 모델 정의
input_dim = 303  # 랜드마크 데이터의 차원
image_shape = (48, 48, 1)  # 이미지 데이터의 형태
z_comb_dim = len(list(itertools.combinations(range(8), 3)))  # z-값 조합의 차원

# 입력 레이어
model_landmark_input = Input(shape=(input_dim,))  # 랜드마크 입력
model_image_input = Input(shape=image_shape)  # 이미지 입력
model_z_input = Input(shape=(z_comb_dim,))  # z-값 조합 입력


# 이미지 서브넷
x_image = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', input_shape=(48,48,1))(model_image_input)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.2)(x_image)  # new dropout layer
x_image = MaxPooling2D((2,2))(x_image)
x_image = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.3)(x_image)  # increased dropout rate
x_image = MaxPooling2D((2,2))(x_image)
x_image = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.2)(x_image)  # new dropout layer
x_image = MaxPooling2D((2,2))(x_image)
x_image = Flatten()(x_image)




x_landmark = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_landmark_input)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)  # Dropout rate 조정

x_landmark = Dense(64, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)

x_landmark = Dense(32, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)

# 이미지 서브넷과 랜드마크 서브넷 합치기
combined_image_landmark = Concatenate()([x_image, x_landmark])

# z-값 서브넷
x_z = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_z_input)
x_z = BatchNormalization()(x_z)
x_z = Activation('elu')(x_z)
x_z = Dropout(0.1)(x_z)

# 이미지 서브넷, 랜드마크 서브넷과 z_value 서브넷 합치기
combined = Concatenate()([combined_image_landmark, x_z])

combined = Dense(128, kernel_regularizer=l2(0.01))(combined)
combined = BatchNormalization()(combined)
combined = Activation('elu')(combined)
combined = Dropout(0.6)(combined)
# 출력 레이어
model_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(combined)

# 모델 정의
model_combined = Model(inputs=[model_image_input, model_landmark_input, model_z_input], outputs=model_output)

# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)


# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)


import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')

plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []
test_landmarks = []
test_z_values = []

y_test = []

for (image, landmark, z_value), label in test_dataset:
    test_images.append(image)
    test_landmarks.append(landmark)
    y_test.extend(label.numpy())
    test_z_values.append(z_value)

y_test_pred = model_combined.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)

# Image  & Z_value & Landmark

In [ ]:
#오후 3시 8분 수정코드 !! 입력 따로 받기 해놓음

# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2

# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}

model_name = 'Image & ZValue & Landmark'

# Define landmark indices
landmark_indices = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46, 300, 293, 334, 296, 336, 285, 295, 282, 283, 276, 33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7, 263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95, 61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 64, 4, 294, 168, 6, 197, 195, 5, 1]
# Define a function to extract selected landmarks
def extract_selected_landmarks(landmarks):
    selected_landmarks = tf.gather(landmarks, indices=landmark_indices, axis=0)
    return tf.reshape(selected_landmarks, [len(landmark_indices) * 3])

def calculate_z_value_combinations(z_values):
    z_combinations = list(itertools.combinations(range(8), 3))
    z_product_combinations = []

    for indices in z_combinations:
        selected_values = tf.gather(z_values, indices)
        product = tf.reduce_prod(selected_values)
        z_product_combinations.append(product)

    return tf.stack(z_product_combinations)

# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "landmarks": tf.io.VarLenFeature(tf.float32),
        "label": tf.io.FixedLenFeature([], tf.int64),
        "z_values": tf.io.FixedLenFeature([8], tf.float32),  # 기본값 수정
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)

    landmarks = tf.sparse.to_dense(parsed_features["landmarks"])
    landmarks = tf.reshape(landmarks, [-1, 3])
    selected_landmarks = extract_selected_landmarks(landmarks)
    selected_landmarks.set_shape([303])

    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    z_values = parsed_features["z_values"]
    z_product_combinations = calculate_z_value_combinations(z_values)

    return (image, selected_landmarks, z_product_combinations), label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)

# z-값 조합의 차원
z_comb_dim = len(list(itertools.combinations(range(8), 3)))

# 모델 정의
input_dim = 303  # 랜드마크 데이터의 차원
image_shape = (48, 48, 1)  # 이미지 데이터의 형태
z_comb_dim = len(list(itertools.combinations(range(8), 3)))  # z-값 조합의 차원

# 입력 레이어
model_landmark_input = Input(shape=(input_dim,))  # 랜드마크 입력
model_image_input = Input(shape=image_shape)  # 이미지 입력
model_z_input = Input(shape=(z_comb_dim,))  # z-값 조합 입력


# 이미지 서브넷
x_image = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', input_shape=(48,48,1))(model_image_input)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.2)(x_image)  # new dropout layer
x_image = MaxPooling2D((2,2))(x_image)
x_image = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.3)(x_image)  # increased dropout rate
x_image = MaxPooling2D((2,2))(x_image)
x_image = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.2)(x_image)  # new dropout layer
x_image = MaxPooling2D((2,2))(x_image)
x_image = Flatten()(x_image)

# z-값 서브넷
x_z = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_z_input)
x_z = BatchNormalization()(x_z)
x_z = Activation('elu')(x_z)
x_z = Dropout(0.1)(x_z)

# 이미지 서브넷과 랜드마크 서브넷 합치기
combined_image_z_value= Concatenate()([x_image, x_z])

# 랜드마크 서브넷
x_landmark = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_landmark_input)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)  # Dropout rate 조정

x_landmark = Dense(64, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)

x_landmark = Dense(32, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)


# 이미지 서브넷, 랜드마크 서브넷과 z_value 서브넷 합치기
combined = Concatenate()([combined_image_z_value, x_landmark])
combined = BatchNormalization()(combined)
combined = Activation('elu')(combined)
combined = Dropout(0.6)(combined)
# 출력 레이어
model_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(combined)

# 모델 정의
model_combined = Model(inputs=[model_image_input, model_landmark_input, model_z_input], outputs=model_output)


# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)


# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)

import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')


plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []
test_landmarks = []
test_z_values = []

y_test = []

for (image, landmark, z_value), label in test_dataset:
    test_images.append(image)
    test_landmarks.append(landmark)
    y_test.extend(label.numpy())
    test_z_values.append(z_value)

y_test_pred = model_combined.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)

#Z_value & Landmark & Image

In [ ]:
#오후 3시 8분 수정코드 !! 입력 따로 받기 해놓음

# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2

# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}

model_name = 'ZValue & Landmark & Image'

# Define landmark indices
landmark_indices = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46, 300, 293, 334, 296, 336, 285, 295, 282, 283, 276, 33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7, 263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95, 61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 64, 4, 294, 168, 6, 197, 195, 5, 1]
# Define a function to extract selected landmarks
def extract_selected_landmarks(landmarks):
    selected_landmarks = tf.gather(landmarks, indices=landmark_indices, axis=0)
    return tf.reshape(selected_landmarks, [len(landmark_indices) * 3])

def calculate_z_value_combinations(z_values):
    z_combinations = list(itertools.combinations(range(8), 3))
    z_product_combinations = []

    for indices in z_combinations:
        selected_values = tf.gather(z_values, indices)
        product = tf.reduce_prod(selected_values)
        z_product_combinations.append(product)

    return tf.stack(z_product_combinations)

# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "landmarks": tf.io.VarLenFeature(tf.float32),
        "label": tf.io.FixedLenFeature([], tf.int64),
        "z_values": tf.io.FixedLenFeature([8], tf.float32),  # 기본값 수정
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)

    landmarks = tf.sparse.to_dense(parsed_features["landmarks"])
    landmarks = tf.reshape(landmarks, [-1, 3])
    selected_landmarks = extract_selected_landmarks(landmarks)
    selected_landmarks.set_shape([303])

    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    z_values = parsed_features["z_values"]
    z_product_combinations = calculate_z_value_combinations(z_values)

    return (image, selected_landmarks, z_product_combinations), label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)

# z-값 조합의 차원
z_comb_dim = len(list(itertools.combinations(range(8), 3)))

# 모델 정의
input_dim = 303  # 랜드마크 데이터의 차원
image_shape = (48, 48, 1)  # 이미지 데이터의 형태
z_comb_dim = len(list(itertools.combinations(range(8), 3)))  # z-값 조합의 차원

# 입력 레이어
model_landmark_input = Input(shape=(input_dim,))  # 랜드마크 입력
model_image_input = Input(shape=image_shape)  # 이미지 입력
model_z_input = Input(shape=(z_comb_dim,))  # z-값 조합 입력

# z-값 서브넷
x_z = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_z_input)
x_z = BatchNormalization()(x_z)
x_z = Activation('elu')(x_z)
x_z = Dropout(0.1)(x_z)

# 랜드마크 서브넷
x_landmark = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_landmark_input)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)  # Dropout rate 조정

x_landmark = Dense(64, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)

x_landmark = Dense(32, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)

# 이미지 서브넷과 랜드마크 서브넷 합치기
combined_landmark_z_value= Concatenate()([x_z, x_landmark])

# 이미지 서브넷
x_image = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', input_shape=(48,48,1))(model_image_input)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.2)(x_image)  # new dropout layer
x_image = MaxPooling2D((2,2))(x_image)
x_image = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.3)(x_image)  # increased dropout rate
x_image = MaxPooling2D((2,2))(x_image)
x_image = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.2)(x_image)  # new dropout layer
x_image = MaxPooling2D((2,2))(x_image)
x_image = Flatten()(x_image)


# 이미지 서브넷, 랜드마크 서브넷과 z_value 서브넷 합치기
combined = Concatenate()([combined_landmark_z_value, x_image])
combined = BatchNormalization()(combined)
combined = Activation('elu')(combined)
combined = Dropout(0.6)(combined)
# 출력 레이어
model_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(combined)

# 모델 정의
model_combined = Model(inputs=[model_image_input, model_landmark_input, model_z_input], outputs=model_output)


# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)


# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)

import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')

plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []
test_landmarks = []
test_z_values = []

y_test = []

for (image, landmark, z_value), label in test_dataset:
    test_images.append(image)
    test_landmarks.append(landmark)
    y_test.extend(label.numpy())
    test_z_values.append(z_value)

y_test_pred = model_combined.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)

# GAN

In [ ]:
# 현재정확도가장높음66.32
# 정규화추가+시각화수정 GAN
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, BatchNormalization, Activation, Dropout, LeakyReLU, GlobalMaxPooling2D, Concatenate, MaxPooling2D, Reshape, Conv2DTranspose
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import itertools
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score, roc_curve, auc
import pandas as pd

model_name = 'GAN'
# 생성된 이미지를 시각화하는 함수 (10 에폭 당 사진 시각화)
def visualize_gen_images(epoch, generator, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, [examples, z_comb_dim])
    landmark_noise = np.random.normal(0, 1, [examples, input_dim])
    img_noise = np.random.normal(0, 1, [examples] + list(image_shape))
    generated_images = generator.predict([img_noise, landmark_noise, noise])

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i, :, :, 0], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan1_0407_generated_image_epoch_%d.png' % epoch)
    plt.close()


# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}

# landmark indices 정의
landmark_indices = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46, 300, 293, 334, 296, 336, 285, 295, 282, 283, 276, 33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7, 263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95, 61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 64, 4, 294, 168, 6, 197, 195, 5, 1]

# landmark 추출 함수 정의
def extract_selected_landmarks(landmarks):
    selected_landmarks = tf.gather(landmarks, indices=landmark_indices, axis=0)
    return tf.reshape(selected_landmarks, [len(landmark_indices) * 3])

def calculate_z_value_combinations(z_values):
    z_combinations = list(itertools.combinations(range(8), 3))
    z_product_combinations = []

    for indices in z_combinations:
        selected_values = tf.gather(z_values, indices)
        product = tf.reduce_prod(selected_values)
        z_product_combinations.append(product)

    return tf.stack(z_product_combinations)

# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "landmarks": tf.io.VarLenFeature(tf.float32),
        "label": tf.io.FixedLenFeature([], tf.int64),
        "z_values": tf.io.FixedLenFeature([8], tf.float32),  # 기본값 수정
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)
    image = tf.cast(image, tf.float32) * (2. / 255) - 1 # 정확도 높이기 위해 추가


    landmarks = tf.sparse.to_dense(parsed_features["landmarks"])
    landmarks = tf.reshape(landmarks, [-1, 3])
    selected_landmarks = extract_selected_landmarks(landmarks)
    selected_landmarks.set_shape([303])

    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    z_values = parsed_features["z_values"]
    z_product_combinations = calculate_z_value_combinations(z_values)

    return (image, selected_landmarks, z_product_combinations), label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)

# 모델 정의
input_dim = 303  # 랜드마크 데이터의 차원
image_shape = (48, 48, 1)  # 이미지 데이터의 형태
z_comb_dim = len(list(itertools.combinations(range(8), 3)))  # z-값 조합의 차원

# 입력 레이어
model_landmark_input = Input(shape=(input_dim,))  # 랜드마크 입력
model_image_input = Input(shape=image_shape)  # 이미지 입력
model_z_input = Input(shape=(z_comb_dim,))  # z-값 조합 입력

def build_generator():
    # 이미지 입력
    img_input = Input(shape=(48,48,1))

    # 랜드마크 입력
    landmark_input = Input(shape=(303,))

    # z-값 조합 입력
    z_comb_input = Input(shape=(56,))

    # 이미지 서브넷
    x_image = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same')(img_input)
    x_image = BatchNormalization()(x_image)
    x_image = Activation('relu')(x_image)
    x_image = Dropout(0.2)(x_image)
    x_image = MaxPooling2D((2,2))(x_image)
    x_image = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
    x_image = BatchNormalization()(x_image)
    x_image = Activation('relu')(x_image)
    x_image = Dropout(0.3)(x_image)
    x_image = MaxPooling2D((2,2))(x_image)
    x_image = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
    x_image = BatchNormalization()(x_image)
    x_image = Activation('relu')(x_image)
    x_image = Dropout(0.2)(x_image)
    x_image = MaxPooling2D((2,2))(x_image)
    x_image = Flatten()(x_image)

    # 랜드마크 서브넷
    x_landmark = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(landmark_input)
    x_landmark = BatchNormalization()(x_landmark)
    x_landmark = Activation('elu')(x_landmark)
    x_landmark = Dropout(0.3)(x_landmark)

    x_landmark = Dense(64, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
    x_landmark = BatchNormalization()(x_landmark)
    x_landmark = Activation('elu')(x_landmark)
    x_landmark = Dropout(0.3)(x_landmark)

    x_landmark = Dense(32, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
    x_landmark = BatchNormalization()(x_landmark)
    x_landmark = Activation('elu')(x_landmark)
    x_landmark = Dropout(0.3)(x_landmark)

    # 이미지 서브넷과 랜드마크 서브넷 합치기
    combined_image_landmark = Concatenate()([x_image, x_landmark])

    # z-값 서브넷
    x_z = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(z_comb_input)
    x_z = BatchNormalization()(x_z)
    x_z = Activation('elu')(x_z)
    x_z = Dropout(0.1)(x_z)

    # 이미지 서브넷, 랜드마크 서브넷과 z_value 서브넷 합치기
    combined = Concatenate()([combined_image_landmark, x_z])
    combined = Dense(128, kernel_regularizer=l2(0.01))(combined)
    combined = BatchNormalization()(combined)
    combined = Activation('elu')(combined)
    combined = Dropout(0.6)(combined)

    # 출력 레이어
    model_output = Dense(np.prod((48,48,1)), activation='tanh')(combined)
    model_output = Reshape((48,48,1))(model_output)

    return Model([img_input, landmark_input, z_comb_input], model_output)



# 판별자 모델 #계속수정중 #아래가 최대 성능 모델_66.32
def build_discriminator():

    # 이미지 입력
    img_input = Input(shape=(48,48,1))

    # 랜드마크 입력
    landmark_input = Input(shape=(303,))

    # z-값 조합 입력
    z_comb_input = Input(shape=(56,))

    # 이미지 서브넷
    x_image = Flatten(input_shape=(48,48,1))(img_input)
    x_image = Dense(128)(x_image)
    x_image = LeakyReLU(alpha=0.2)(x_image)

    # 랜드마크 서브넷
    x_landmark = Dense(101)(landmark_input)
    x_landmark = LeakyReLU(alpha=0.2)(x_landmark)

    # z-값 서브넷
    x_z = Dense(101)(z_comb_input)
    x_z = LeakyReLU(alpha=0.2)(x_z)

    # 이미지 서브넷, 랜드마크 서브넷과 z_value 서브넷 합치기
    combined = Concatenate()([x_image, x_landmark, x_z])

    combined = Dense(256)(combined)
    combined = LeakyReLU(alpha=0.2)(combined)

    # 출력 레이어
    model_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(combined)

    return Model([img_input, landmark_input, z_comb_input], model_output)


# Optimizer
optimizer = Adam(0.0002, 0.5)

# 판별자 컴파일
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# 생성자 생성
generator = build_generator()

# 이미지 입력
img_input = Input(shape=(48,48,1))

# 랜드마크 입력
landmark_input = Input(shape=(303,))

# z-값 조합 입력
z_comb_input = Input(shape=(56,))

# 생성자 모델에 입력 전달
gen_imgs = generator([img_input, landmark_input, z_comb_input])

# 판별자 모델에 이미지 전달
valid = discriminator([gen_imgs, landmark_input, z_comb_input])

# 결합된 모델 (생성자와 판별자를 쌓음)
model_combined = Model([img_input, landmark_input, z_comb_input], valid)

# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)


# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)
import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')


plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []
test_landmarks = []
test_z_values = []

y_test = []

for (image, landmark, z_value), label in test_dataset:
    test_images.append(image)
    test_landmarks.append(landmark)
    y_test.extend(label.numpy())
    test_z_values.append(z_value)

y_test_pred = model_combined.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)


# 1-d

In [ ]:
# dropout조정하고 dense를 줄임
#과적합다시줄여야할듯

# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc

# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}
model_name = '1-d'
# Define landmark indices
landmark_indices = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46, 300, 293, 334, 296, 336, 285, 295, 282, 283, 276, 33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7, 263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95, 61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 64, 4, 294, 168, 6, 197, 195, 5, 1]
# Define a function to extract selected landmarks
def extract_selected_landmarks(landmarks):
    selected_landmarks = tf.gather(landmarks, indices=landmark_indices, axis=0)
    return tf.reshape(selected_landmarks, [len(landmark_indices) * 3])

def calculate_z_value_combinations(z_values):
    z_combinations = list(itertools.combinations(range(8), 3))
    z_product_combinations = []

    for indices in z_combinations:
        selected_values = tf.gather(z_values, indices)
        product = tf.reduce_prod(selected_values)
        z_product_combinations.append(product)

    return tf.stack(z_product_combinations)

# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "landmarks": tf.io.VarLenFeature(tf.float32),
        "label": tf.io.FixedLenFeature([], tf.int64),
        "z_values": tf.io.FixedLenFeature([8], tf.float32),  # 기본값 수정
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)

    landmarks = tf.sparse.to_dense(parsed_features["landmarks"])
    landmarks = tf.reshape(landmarks, [-1, 3])
    selected_landmarks = extract_selected_landmarks(landmarks)
    selected_landmarks.set_shape([303])

    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    z_values = parsed_features["z_values"]
    z_product_combinations = calculate_z_value_combinations(z_values)

    return (image, selected_landmarks, z_product_combinations), label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)

# z-값 조합의 차원
z_comb_dim = len(list(itertools.combinations(range(8), 3)))

# 모델 정의
input_dim = 303  # 랜드마크 데이터의 차원
image_shape = (48, 48, 1)  # 이미지 데이터의 형태
z_comb_dim = len(list(itertools.combinations(range(8), 3)))  # z-값 조합의 차원

# 입력 레이어
model_landmark_input = Input(shape=(input_dim,))  # 랜드마크 입력
model_image_input = Input(shape=image_shape)  # 이미지 입력
model_z_input = Input(shape=(z_comb_dim,))  # z-값 조합 입력


# 이미지 서브넷 간소화
x_image = Conv2D(16, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(0.01))(model_image_input)  # 필터 수 감소 및 l2 정규화 계수 감소
x_image = MaxPooling2D(pool_size=(2, 2))(x_image)
x_image = Flatten()(x_image)
x_image = Dense(16, activation='relu')(x_image)  # 차원 축소를 위한 추가 레이어

# 랜드마크 서브넷 간소화
x_landmark = Dense(50, kernel_regularizer=l2(0.01))(model_landmark_input)  # 뉴런 수 감소 및 l2 정규화 계수 감소
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.7)(x_landmark)  # 드롭아웃 비율 조정
x_landmark = Dense(16, activation='relu')(x_landmark)  # 차원 축소를 위한 추가 레이어

# 이미지 서브넷과 랜드마크 서브넷 합치기
combined_image_landmark = Concatenate()([x_image, x_landmark])

# z-값 서브넷 간소화
x_z = Dense(50, kernel_regularizer=l2(0.01))(model_z_input)  # 뉴런 수 감소
x_z = BatchNormalization()(x_z)
x_z = Activation('elu')(x_z)
x_z = Dropout(0.7)(x_z)
x_z = Dense(16, activation='relu')(x_z)  # 차원 축소를 위한 추가 레이어

# 이미지 서브넷, 랜드마크 서브넷과 z_value 서브넷 합치기
combined = Concatenate()([combined_image_landmark, x_z])

combined = Dense(128, kernel_regularizer=l2(0.01))(combined)
combined = BatchNormalization()(combined)
combined = Activation('elu')(combined)
combined = Dropout(0.65)(combined)

# 출력 레이어
model_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(combined)

# 모델 정의
model_combined = Model(inputs=[model_image_input, model_landmark_input, model_z_input], outputs=model_output)

# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)


# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)

import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')

plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []
test_landmarks = []
test_z_values = []

y_test = []

for (image, landmark, z_value), label in test_dataset:
    test_images.append(image)
    test_landmarks.append(landmark)
    y_test.extend(label.numpy())
    test_z_values.append(z_value)

y_test_pred = model_combined.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)


# Freezing

In [ ]:



# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard

# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}
model_name = 'freezing'
# Define landmark indices
landmark_indices = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46, 300, 293, 334, 296, 336, 285, 295, 282, 283, 276, 33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7, 263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95, 61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 64, 4, 294, 168, 6, 197, 195, 5, 1]
# Define a function to extract selected landmarks
def extract_selected_landmarks(landmarks):
    selected_landmarks = tf.gather(landmarks, indices=landmark_indices, axis=0)
    return tf.reshape(selected_landmarks, [len(landmark_indices) * 3])

def calculate_z_value_combinations(z_values):
    z_combinations = list(itertools.combinations(range(8), 3))
    z_product_combinations = []

    for indices in z_combinations:
        selected_values = tf.gather(z_values, indices)
        product = tf.reduce_prod(selected_values)
        z_product_combinations.append(product)

    return tf.stack(z_product_combinations)

# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "landmarks": tf.io.VarLenFeature(tf.float32),
        "label": tf.io.FixedLenFeature([], tf.int64),
        "z_values": tf.io.FixedLenFeature([8], tf.float32),  # 기본값 수정
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)

    landmarks = tf.sparse.to_dense(parsed_features["landmarks"])
    landmarks = tf.reshape(landmarks, [-1, 3])
    selected_landmarks = extract_selected_landmarks(landmarks)
    selected_landmarks.set_shape([303])

    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    z_values = parsed_features["z_values"]
    z_product_combinations = calculate_z_value_combinations(z_values)

    return (image, selected_landmarks, z_product_combinations), label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed

train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)

def extract_image(inputs, label):
    return inputs[0], label

def extract_landmark(inputs, label):
    return inputs[1], label

def extract_z_value(inputs, label):
    return inputs[2], label

# 이미지 데이터셋
train_image_dataset = train_dataset.map(extract_image)
val_image_dataset = val_dataset.map(extract_image)
test_image_dataset = test_dataset.map(extract_image)

# 랜드마크 데이터셋
train_landmark_dataset = train_dataset.map(extract_landmark)
val_landmark_dataset = val_dataset.map(extract_landmark)
test_landmark_dataset = test_dataset.map(extract_landmark)

# z_value 데이터셋
train_z_dataset = train_dataset.map(extract_z_value)
val_z_dataset = val_dataset.map(extract_z_value)
test_z_dataset = test_dataset.map(extract_z_value)


# z-값 조합의 차원
z_comb_dim = len(list(itertools.combinations(range(8), 3)))

# 모델 정의
input_dim = 303  # 랜드마크 데이터의 차원
image_shape = (48, 48, 1)  # 이미지 데이터의 형태
z_comb_dim = len(list(itertools.combinations(range(8), 3)))  # z-값 조합의 차원

# 입력 레이어
model_landmark_input = Input(shape=(input_dim,))  # 랜드마크 입력
model_image_input = Input(shape=image_shape)  # 이미지 입력
model_z_input = Input(shape=(z_comb_dim,))  # z-값 조합 입력


# 이미지 서브넷
x_image = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', input_shape=(48,48,1))(model_image_input)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.2)(x_image)  # new dropout layer
x_image = MaxPooling2D((2,2))(x_image)
x_image = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.3)(x_image)  # increased dropout rate
x_image = MaxPooling2D((2,2))(x_image)
x_image = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.2)(x_image)  # new dropout layer
x_image = MaxPooling2D((2,2))(x_image)
x_image = Flatten()(x_image)

# 랜드마크 서브넷
x_landmark = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_landmark_input)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)  # Dropout rate 조정

x_landmark = Dense(64, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)

x_landmark = Dense(32, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)

# 이미지 서브넷과 랜드마크 서브넷 합치기
combined_image_landmark = Concatenate()([x_image, x_landmark])

# z-값 서브넷
x_z = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_z_input)
x_z = BatchNormalization()(x_z)
x_z = Activation('elu')(x_z)
x_z = Dropout(0.1)(x_z)

# 이미지 서브넷, 랜드마크 서브넷과 z_value 서브넷 합치기
combined = Concatenate()([combined_image_landmark, x_z])

combined = Dense(128, kernel_regularizer=l2(0.01))(combined)
combined = BatchNormalization()(combined)
combined = Activation('elu')(combined)
combined = Dropout(0.6)(combined)
# 출력 레이어
model_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(combined)

# 모델 정의

# 모델 컴파일

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)


# 모델 훈련
# 이미지 서브넷 정의 및 학습
image_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(x_image)
model_image = Model(inputs=model_image_input, outputs=image_output)
model_image.compile(optimizer=Adam(learning_rate=0.0001), loss=focal_loss(), metrics=['accuracy'])
model_image.fit(x=train_image_dataset, epochs=epochs, validation_data=val_image_dataset, callbacks=[early_stopping, reduce_lr])

# 랜드마크 서브넷 정의 및 학습
landmark_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(x_landmark)
model_landmark = Model(inputs=model_landmark_input, outputs=landmark_output)
model_landmark.compile(optimizer=Adam(learning_rate=0.0001), loss=focal_loss(), metrics=['accuracy'])
model_landmark.fit(x=train_landmark_dataset, epochs=epochs, validation_data=val_landmark_dataset, callbacks=[early_stopping, reduce_lr])

# z-값 서브넷 정의 및 학습
z_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(x_z)
model_z = Model(inputs=model_z_input, outputs=z_output)
model_z.compile(optimizer=Adam(learning_rate=0.0001), loss=focal_loss(), metrics=['accuracy'])
model_z.fit(x=train_z_dataset, epochs=epochs, validation_data=val_z_dataset, callbacks=[early_stopping, reduce_lr])


# 서브넷 가중치 동결
for layer in model_image.layers:
    layer.trainable = False

for layer in model_landmark.layers:
    layer.trainable = False

for layer in model_z.layers:
    layer.trainable = False

# 전체 모델 정의 및 학습

model_combined = Model(inputs=[model_image_input, model_landmark_input, model_z_input], outputs=model_output)


# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback


# Early stopping callback


# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)



import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')


plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []
test_landmarks = []
test_z_values = []

y_test = []

for (image, landmark, z_value), label in test_dataset:
    test_images.append(image)
    test_landmarks.append(landmark)
    y_test.extend(label.numpy())
    test_z_values.append(z_value)

y_test_pred = model_combined.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)


# weighted

In [ ]:

# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score, roc_curve, auc
import pandas as pd
from keras.callbacks import ReduceLROnPlateau

# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}
model_name = 'weighted'
# Define landmark indices
landmark_indices = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46, 300, 293, 334, 296, 336, 285, 295, 282, 283, 276, 33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7, 263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95, 61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 64, 4, 294, 168, 6, 197, 195, 5, 1]
# Define a function to extract selected landmarks
def extract_selected_landmarks(landmarks):
    selected_landmarks = tf.gather(landmarks, indices=landmark_indices, axis=0)
    return tf.reshape(selected_landmarks, [len(landmark_indices) * 3])

def calculate_z_value_combinations(z_values):
    z_combinations = list(itertools.combinations(range(8), 3))
    z_product_combinations = []

    for indices in z_combinations:
        selected_values = tf.gather(z_values, indices)
        product = tf.reduce_prod(selected_values)
        z_product_combinations.append(product)

    return tf.stack(z_product_combinations)

# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "landmarks": tf.io.VarLenFeature(tf.float32),
        "label": tf.io.FixedLenFeature([], tf.int64),
        "z_values": tf.io.FixedLenFeature([8], tf.float32),  # 기본값 수정
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)

    landmarks = tf.sparse.to_dense(parsed_features["landmarks"])
    landmarks = tf.reshape(landmarks, [-1, 3])
    selected_landmarks = extract_selected_landmarks(landmarks)
    selected_landmarks.set_shape([303])

    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    z_values = parsed_features["z_values"]
    z_product_combinations = calculate_z_value_combinations(z_values)

    return (image, selected_landmarks, z_product_combinations), label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)

def extract_image(inputs, label):
    return inputs[0], label

def extract_landmark(inputs, label):
    return inputs[1], label

def extract_z_value(inputs, label):
    return inputs[2], label

# 이미지 데이터셋
train_image_dataset = train_dataset.map(extract_image)
val_image_dataset = val_dataset.map(extract_image)
test_image_dataset = test_dataset.map(extract_image)

# 랜드마크 데이터셋
train_landmark_dataset = train_dataset.map(extract_landmark)
val_landmark_dataset = val_dataset.map(extract_landmark)
test_landmark_dataset = test_dataset.map(extract_landmark)

# z_value 데이터셋
train_z_dataset = train_dataset.map(extract_z_value)
val_z_dataset = val_dataset.map(extract_z_value)
test_z_dataset = test_dataset.map(extract_z_value)


# z-값 조합의 차원
z_comb_dim = len(list(itertools.combinations(range(8), 3)))

# 모델 정의
input_dim = 303  # 랜드마크 데이터의 차원
image_shape = (48, 48, 1)  # 이미지 데이터의 형태
z_comb_dim = len(list(itertools.combinations(range(8), 3)))  # z-값 조합의 차원

# 입력 레이어
model_landmark_input = Input(shape=(input_dim,))  # 랜드마크 입력
model_image_input = Input(shape=image_shape)  # 이미지 입력
model_z_input = Input(shape=(z_comb_dim,))  # z-값 조합 입력


def build_emotion_classifier():
    input_dim = 303  # 랜드마크 데이터의 차원
    image_shape = (48, 48, 1)  # 이미지 데이터의 형태
    z_comb_dim = len(list(itertools.combinations(range(8), 3)))

    # 입력 레이어
    model_landmark_input = Input(shape=(input_dim,))  # 랜드마크 입력
    model_image_input = Input(shape=image_shape)  # 이미지 입력
    model_z_input = Input(shape=(z_comb_dim,))  # z-값 조합 입력

    # 이미지 서브넷
    x_image = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', input_shape=(48,48,1))(model_image_input)
    x_image = BatchNormalization(axis=3)(x_image)
    x_image = Activation('relu')(x_image)
    x_image = Dropout(0.3)(x_image)  # new dropout layer
    x_image = MaxPooling2D((2,2))(x_image)
    x_image = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
    x_image = BatchNormalization(axis=3)(x_image)
    x_image = Activation('relu')(x_image)
    x_image = Dropout(0.3)(x_image)  # increased dropout rate
    x_image = MaxPooling2D((2,2))(x_image)
    x_image = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
    x_image = BatchNormalization(axis=3)(x_image)
    x_image = Activation('relu')(x_image)
    x_image = Dropout(0.3)(x_image)  # new dropout layer
    x_image = MaxPooling2D((2,2))(x_image)
    x_image = Flatten()(x_image)


    # 랜드마크 서브넷
    x_landmark = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_landmark_input)
    x_landmark = BatchNormalization()(x_landmark)
    x_landmark = Activation('elu')(x_landmark)
    x_landmark = Dropout(0.3)(x_landmark)  # Dropout rate 조정

    x_landmark = Dense(64, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
    x_landmark = BatchNormalization()(x_landmark)
    x_landmark = Activation('elu')(x_landmark)
    x_landmark = Dropout(0.3)(x_landmark)

    x_landmark = Dense(32, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
    x_landmark = BatchNormalization()(x_landmark)
    x_landmark = Activation('elu')(x_landmark)
    x_landmark = Dropout(0.3)(x_landmark)

    # 이미지 서브넷과 랜드마크 서브넷 합치기
    combined_image_landmark = Concatenate()([x_image, x_landmark])

    # z-값 서브넷
    x_z = Dense(101, kernel_regularizer=l2(0.01))(model_z_input)
    x_z = BatchNormalization()(x_z)
    x_z = Activation('elu')(x_z)
    x_z = Dropout(0.5)(x_z)

    # 이미지 서브넷, 랜드마크 서브넷과 z_value 서브넷 합치기
    combined = Concatenate()([combined_image_landmark, x_z])

    combined = Dense(128, kernel_regularizer=l2(0.01))(combined)
    combined = BatchNormalization()(combined)
    combined = Activation('elu')(combined)
    combined = Dropout(0.6)(combined)
    # 출력 레이어
    model_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(combined)

    # 모델 정의
    model_combined = Model(inputs=[model_image_input, model_landmark_input, model_z_input], outputs=model_output)

    # 모델 컴파일
    def focal_loss(gamma=2.0, alpha=0.25):
        def focal_loss_fixed(y_true, y_pred):
            # y_true: 실제 레이블, y_pred: 모델 예측
            pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
            pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
            return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

        return focal_loss_fixed

    model_combined.compile(
        optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
        loss=focal_loss(),
        metrics=['accuracy'])

    # 모델 요약
    model_combined.summary()

    # Early stopping callback


    # 이미지 서브넷 정의
    image_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(x_image)
    model_image = Model(inputs=model_image_input, outputs=image_output)

    # 랜드마크 서브넷 정의
    landmark_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(x_landmark)
    model_landmark = Model(inputs=model_landmark_input, outputs=landmark_output)

    # z-값 서브넷 정의
    z_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(x_z)
    model_z = Model(inputs=model_z_input, outputs=z_output)

    return model_combined, model_image, model_landmark, model_z

def freeze_weights(model):
    for layer in model.layers:
        layer.trainable = False

def unfreeze_weights(model):
    for layer in model.layers:
        layer.trainable = True

emotion_classifier, model_image, model_landmark, model_z = build_emotion_classifier()

# Early stopping callback


# Focal loss function
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        return -tf.reduce_sum(alpha * tf.pow(1. - pt, gamma) * tf.math.log(pt))
    return focal_loss_fixed

early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)


freeze_weights(model_landmark)
freeze_weights(model_z)
model_image.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),
    loss=focal_loss(),
    metrics=['accuracy'])
model_image.summary()  # Add this line to print the summary



model_image.fit(x=train_image_dataset, epochs=epochs, validation_data=val_image_dataset, callbacks=[early_stopping, reduce_lr])

# Compile and train the landmark subnetwork
unfreeze_weights(model_landmark)
freeze_weights(model_image)
freeze_weights(model_z)
model_landmark.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),
    loss=focal_loss(),
    metrics=['accuracy'])
model_landmark.summary()  # Add this line to print the summary

model_landmark.fit(x=train_landmark_dataset, epochs=epochs, validation_data=val_landmark_dataset, callbacks=[early_stopping, reduce_lr])

# Compile and train the z-value subnetwork
unfreeze_weights(model_z)
freeze_weights(model_image)
freeze_weights(model_landmark)
model_z.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),
    loss=focal_loss(),
    metrics=['accuracy'])
model_z.summary()  # Add this line to print the summary

model_z.fit(x=train_z_dataset, epochs=epochs, validation_data=val_z_dataset, callbacks=[early_stopping, reduce_lr])




# Unfreeze all weights for combined training
unfreeze_weights(model_image)
unfreeze_weights(model_landmark)
unfreeze_weights(model_z)

# Compile the combined model
emotion_classifier.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),
    loss=focal_loss(),
    metrics=['accuracy'])
emotion_classifier.summary()

# Train the combined model
history = emotion_classifier.fit(
    x=train_dataset,
    epochs=epochs,
    validation_data=val_dataset,
    callbacks=[early_stopping, reduce_lr]
)



import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')


plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = emotion_classifier.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []
test_landmarks = []
test_z_values = []

y_test = []

for (image, landmark, z_value), label in test_dataset:
    test_images.append(image)
    test_landmarks.append(landmark)
    y_test.extend(label.numpy())
    test_z_values.append(z_value)

y_test_pred = emotion_classifier.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = emotion_classifier.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

emotion_classifier.save(model_save_path_h5)
emotion_classifier.save(model_save_path_keras)


In [ ]:


# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2
from sklearn.metrics import roc_auc_score, precision_recall_curve, average_precision_score, roc_curve, auc
import pandas as pd
from keras.callbacks import ReduceLROnPlateau

# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}
model_name = 'weighted'
# Define landmark indices
landmark_indices = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46, 300, 293, 334, 296, 336, 285, 295, 282, 283, 276, 33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7, 263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95, 61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 64, 4, 294, 168, 6, 197, 195, 5, 1]
# Define a function to extract selected landmarks
def extract_selected_landmarks(landmarks):
    selected_landmarks = tf.gather(landmarks, indices=landmark_indices, axis=0)
    return tf.reshape(selected_landmarks, [len(landmark_indices) * 3])

def calculate_z_value_combinations(z_values):
    z_combinations = list(itertools.combinations(range(8), 3))
    z_product_combinations = []

    for indices in z_combinations:
        selected_values = tf.gather(z_values, indices)
        product = tf.reduce_prod(selected_values)
        z_product_combinations.append(product)

    return tf.stack(z_product_combinations)

# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "landmarks": tf.io.VarLenFeature(tf.float32),
        "label": tf.io.FixedLenFeature([], tf.int64),
        "z_values": tf.io.FixedLenFeature([8], tf.float32),  # 기본값 수정
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)

    landmarks = tf.sparse.to_dense(parsed_features["landmarks"])
    landmarks = tf.reshape(landmarks, [-1, 3])
    selected_landmarks = extract_selected_landmarks(landmarks)
    selected_landmarks.set_shape([303])

    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    z_values = parsed_features["z_values"]
    z_product_combinations = calculate_z_value_combinations(z_values)

    return (image, selected_landmarks, z_product_combinations), label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)

def extract_image(inputs, label):
    return inputs[0], label

def extract_landmark(inputs, label):
    return inputs[1], label

def extract_z_value(inputs, label):
    return inputs[2], label

# 이미지 데이터셋
train_image_dataset = train_dataset.map(extract_image)
val_image_dataset = val_dataset.map(extract_image)
test_image_dataset = test_dataset.map(extract_image)

# 랜드마크 데이터셋
train_landmark_dataset = train_dataset.map(extract_landmark)
val_landmark_dataset = val_dataset.map(extract_landmark)
test_landmark_dataset = test_dataset.map(extract_landmark)

# z_value 데이터셋
train_z_dataset = train_dataset.map(extract_z_value)
val_z_dataset = val_dataset.map(extract_z_value)
test_z_dataset = test_dataset.map(extract_z_value)


# z-값 조합의 차원
z_comb_dim = len(list(itertools.combinations(range(8), 3)))

# 모델 정의
input_dim = 303  # 랜드마크 데이터의 차원
image_shape = (48, 48, 1)  # 이미지 데이터의 형태
z_comb_dim = len(list(itertools.combinations(range(8), 3)))  # z-값 조합의 차원

# 입력 레이어
model_landmark_input = Input(shape=(input_dim,))  # 랜드마크 입력
model_image_input = Input(shape=image_shape)  # 이미지 입력
model_z_input = Input(shape=(z_comb_dim,))  # z-값 조합 입력



# Custom layer to combine tensors with two learnable scalars
class CombineLayer(Layer):
    def __init__(self, **kwargs):
        super(CombineLayer, self).__init__(**kwargs)
        self.a = None
        self.b = None

    def build(self, input_shape):
        self.a = self.add_weight(shape=(1,), initializer="uniform", trainable=True, name="a")
        self.b = self.add_weight(shape=(1,), initializer="uniform", trainable=True, name="b")

    def call(self, inputs):
        A, B = inputs
        return (1 - self.a) * A + self.a * B + (1 - self.b) * A * B

def build_model():
    input_dim = 303  # Dimension of landmark data
    image_shape = (48, 48, 1)  # Shape of image data
    z_comb_dim = len(list(itertools.combinations(range(8), 3)))  # Dimension of z-combination

    # Input layers
    model_landmark_input = Input(shape=(input_dim,))
    model_image_input = Input(shape=image_shape)
    model_z_input = Input(shape=(z_comb_dim,))

    # Image Subnet
    x_image = Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same')(model_image_input)
    x_image = BatchNormalization(axis=3)(x_image)
    x_image = Activation('relu')(x_image)
    x_image = Dropout(0.1)(x_image)
    x_image = MaxPooling2D((2, 2))(x_image)

    x_image = Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same')(x_image)
    x_image = BatchNormalization(axis=3)(x_image)
    x_image = Activation('relu')(x_image)
    x_image = Dropout(0.1)(x_image)
    x_image = MaxPooling2D((2, 2))(x_image)

    x_image = Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same')(x_image)
    x_image = BatchNormalization(axis=3)(x_image)
    x_image = Activation('relu')(x_image)
    x_image = Dropout(0.4)(x_image)
    x_image = MaxPooling2D((2, 2))(x_image)
    x_image = Flatten()(x_image)

    # Landmark Subnet
    x_landmark = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_landmark_input)
    x_landmark = BatchNormalization()(x_landmark)
    x_landmark = Activation('elu')(x_landmark)
    x_landmark = Dropout(0.5)(x_landmark)

    x_landmark = Dense(64, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
    x_landmark = BatchNormalization()(x_landmark)
    x_landmark = Activation('elu')(x_landmark)
    x_landmark = Dropout(0.2)(x_landmark)

    x_landmark = Dense(32, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
    x_landmark = BatchNormalization()(x_landmark)
    x_landmark = Activation('elu')(x_landmark)
    x_landmark = Dropout(0.2)(x_landmark)

    # Resize x_image to match x_landmark
    x_image_resized = Dense(32)(x_image)

    # Combine Image and Landmark Subnets using CombineLayer
    combined_image_landmark = CombineLayer(name="combine_layer_1")([x_image_resized, x_landmark])

    # z-value Subnet
    x_z = Dense(101, kernel_regularizer=l2(0.01))(model_z_input)
    x_z = BatchNormalization()(x_z)
    x_z = Activation('elu')(x_z)
    x_z = Dropout(0.5)(x_z)

    # Resize combined_image_landmark to match x_z
    combined_image_landmark_resized = Dense(101)(combined_image_landmark)

    # Combine all Subnets using CombineLayer
    combined = CombineLayer(name="combine_layer_2")([combined_image_landmark_resized, x_z])
    combined = Dense(128, kernel_regularizer=l2(0.01))(combined)
    combined = BatchNormalization()(combined)
    combined = Activation('relu')(combined)
    combined = Dropout(0.5)(combined)

    # Output layer
    emotion_labels = 7  # Adjust this to match the number of emotion categories
    model_output = Dense(emotion_labels, activation='softmax', kernel_regularizer=l2(0.01))(combined)

    # Define the combined model
    model_combined = Model(inputs=[model_image_input, model_landmark_input, model_z_input], outputs=model_output)

    # Compile the model
    learning_rate = 0.00016
    model_combined.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )


    # 모델 요약
    model_combined.summary()

    # Early stopping callback


    # 이미지 서브넷 정의
    image_output = Dense(7, activation='softmax', kernel_regularizer=l2(0.01))(x_image)
    model_image = Model(inputs=model_image_input, outputs=image_output)

    # 랜드마크 서브넷 정의
    landmark_output = Dense(7, activation='softmax', kernel_regularizer=l2(0.01))(x_landmark)
    model_landmark = Model(inputs=model_landmark_input, outputs=landmark_output)

    # z-값 서브넷 정의
    z_output = Dense(7, activation='softmax', kernel_regularizer=l2(0.01))(x_z)
    model_z = Model(inputs=model_z_input, outputs=z_output)

    return model_combined, model_image, model_landmark, model_z

def freeze_weights(model):
    for layer in model.layers:
        layer.trainable = False

def unfreeze_weights(model):
    for layer in model.layers:
        layer.trainable = True

emotion_classifier, model_image, model_landmark, model_z = build_model()

# Early stopping callback


# Focal loss function
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        return -tf.reduce_sum(alpha * tf.pow(1. - pt, gamma) * tf.math.log(pt))
    return focal_loss_fixed

early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)


freeze_weights(model_landmark)
freeze_weights(model_z)
model_image.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),
    loss=focal_loss(),
    metrics=['accuracy'])
model_image.summary()  # Add this line to print the summary



model_image.fit(x=train_image_dataset, epochs=epochs, validation_data=val_image_dataset, callbacks=[early_stopping, reduce_lr])

# Compile and train the landmark subnetwork
unfreeze_weights(model_landmark)
freeze_weights(model_image)
freeze_weights(model_z)
model_landmark.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),
    loss=focal_loss(),
    metrics=['accuracy'])
model_landmark.summary()  # Add this line to print the summary

model_landmark.fit(x=train_landmark_dataset, epochs=epochs, validation_data=val_landmark_dataset, callbacks=[early_stopping, reduce_lr])

# Compile and train the z-value subnetwork
unfreeze_weights(model_z)
freeze_weights(model_image)
freeze_weights(model_landmark)
model_z.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),
    loss=focal_loss(),
    metrics=['accuracy'])
model_z.summary()  # Add this line to print the summary

model_z.fit(x=train_z_dataset, epochs=epochs, validation_data=val_z_dataset, callbacks=[early_stopping, reduce_lr])




# Unfreeze all weights for combined training
unfreeze_weights(model_image)
unfreeze_weights(model_landmark)
unfreeze_weights(model_z)

# Compile the combined model
emotion_classifier.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),
    loss=focal_loss(),
    metrics=['accuracy'])
emotion_classifier.summary()

# Train the combined model
history = emotion_classifier.fit(
    x=train_dataset,
    epochs=epochs,
    validation_data=val_dataset,
    callbacks=[early_stopping, reduce_lr]
)



import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')


plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = emotion_classifier.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []
test_landmarks = []
test_z_values = []

y_test = []

for (image, landmark, z_value), label in test_dataset:
    test_images.append(image)
    test_landmarks.append(landmark)
    y_test.extend(label.numpy())
    test_z_values.append(z_value)

y_test_pred = emotion_classifier.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = emotion_classifier.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

emotion_classifier.save(model_save_path_h5)
emotion_classifier.save(model_save_path_keras)


In [ ]:
test_loss, test_accuracy = emotion_classifier.evaluate(test_dataset, verbose=1)

# Print the results
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 기본형 Image & Z_valuse & Landmark

In [ ]:
#오후 3시 8분 수정코드 !! 입력 따로 받기 해놓음


# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2

# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}
model_name = 'basic 3 input'
# Define landmark indices
landmark_indices = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46, 300, 293, 334, 296, 336, 285, 295, 282, 283, 276, 33, 246, 161, 160, 159, 158, 157, 173, 133, 155, 154, 153, 145, 144, 163, 7, 263, 466, 388, 387, 386, 385, 384, 398, 362, 382, 381, 380, 374, 373, 390, 249, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95, 61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 64, 4, 294, 168, 6, 197, 195, 5, 1]
# Define a function to extract selected landmarks
def extract_selected_landmarks(landmarks):
    selected_landmarks = tf.gather(landmarks, indices=landmark_indices, axis=0)
    return tf.reshape(selected_landmarks, [len(landmark_indices) * 3])

def calculate_z_value_combinations(z_values):
    z_combinations = list(itertools.combinations(range(8), 3))
    z_product_combinations = []

    for indices in z_combinations:
        selected_values = tf.gather(z_values, indices)
        product = tf.reduce_prod(selected_values)
        z_product_combinations.append(product)

    return tf.stack(z_product_combinations)

# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "landmarks": tf.io.VarLenFeature(tf.float32),
        "label": tf.io.FixedLenFeature([], tf.int64),
        "z_values": tf.io.FixedLenFeature([8], tf.float32),  # 기본값 수정
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)

    landmarks = tf.sparse.to_dense(parsed_features["landmarks"])
    landmarks = tf.reshape(landmarks, [-1, 3])
    selected_landmarks = extract_selected_landmarks(landmarks)
    selected_landmarks.set_shape([303])

    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    z_values = parsed_features["z_values"]
    z_product_combinations = calculate_z_value_combinations(z_values)

    return (image, selected_landmarks, z_product_combinations), label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)

# z-값 조합의 차원
z_comb_dim = len(list(itertools.combinations(range(8), 3)))

# 모델 정의
input_dim = 303  # 랜드마크 데이터의 차원
image_shape = (48, 48, 1)  # 이미지 데이터의 형태
z_comb_dim = len(list(itertools.combinations(range(8), 3)))  # z-값 조합의 차원

# 입력 레이어
model_landmark_input = Input(shape=(input_dim,))  # 랜드마크 입력
model_image_input = Input(shape=image_shape)  # 이미지 입력
model_z_input = Input(shape=(z_comb_dim,))  # z-값 조합 입력


# 이미지 서브넷
x_image = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same', input_shape=(48,48,1))(model_image_input)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.2)(x_image)  # new dropout layer
x_image = MaxPooling2D((2,2))(x_image)
x_image = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.3)(x_image)  # increased dropout rate
x_image = MaxPooling2D((2,2))(x_image)
x_image = Conv2D(128, kernel_size=(3,3), strides=(1,1), padding='same')(x_image)
x_image = BatchNormalization(axis=3)(x_image)
x_image = Activation('relu')(x_image)
x_image = Dropout(0.2)(x_image)  # new dropout layer
x_image = MaxPooling2D((2,2))(x_image)
x_image = Flatten()(x_image)

# 랜드마크 서브넷
x_landmark = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_landmark_input)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)  # Dropout rate 조정

x_landmark = Dense(64, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)

x_landmark = Dense(32, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(x_landmark)
x_landmark = BatchNormalization()(x_landmark)
x_landmark = Activation('elu')(x_landmark)
x_landmark = Dropout(0.3)(x_landmark)

# 이미지 서브넷과 랜드마크 서브넷 합치기
combined_image_landmark = Concatenate()([x_image, x_landmark])

# z-값 서브넷
x_z = Dense(101, kernel_regularizer=l2(0.001), kernel_initializer='he_normal')(model_z_input)
x_z = BatchNormalization()(x_z)
x_z = Activation('elu')(x_z)
x_z = Dropout(0.1)(x_z)

# 이미지 서브넷, 랜드마크 서브넷과 z_value 서브넷 합치기
combined = Concatenate()([combined_image_landmark, x_z])

combined = Dense(128, kernel_regularizer=l2(0.01))(combined)
combined = BatchNormalization()(combined)
combined = Activation('elu')(combined)
combined = Dropout(0.6)(combined)
# 출력 레이어
model_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(combined)

# 모델 정의
model_combined = Model(inputs=[model_image_input, model_landmark_input, model_z_input], outputs=model_output)

# 모델 컴파일
# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)

# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)

import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')


plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []
test_landmarks = []
test_z_values = []

y_test = []

for (image, landmark, z_value), label in test_dataset:
    test_images.append(image)
    test_landmarks.append(landmark)
    y_test.extend(label.numpy())
    test_z_values.append(z_value)

y_test_pred = model_combined.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_images), np.vstack(test_landmarks), np.vstack(test_z_values)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)


# VGG16

In [ ]:

# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Lambda

# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}
model_name = 'vggnet16'


# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)


    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    return image, label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)


# 모델 정의
image_shape = (48, 48, 1)  # 이미지 데이터의 형태

model_image_input = Input(shape=image_shape)  # 이미지 입력


# 입력 레이어
# 랜드마크와 유클리드 거리 서브넷

# 이미지 서브넷
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Flatten, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

# Assuming emotion_labels is defined somewhere


# Concatenate grayscale image to RGB
x_image = Lambda(lambda x: tf.concat([x, x, x], axis=-1))(model_image_input)

# Load VGG16 model pre-trained on ImageNet, excluding top layers
vggnet_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Freeze VGG16 layers
for layer in vggnet_model.layers:
    layer.trainable = False

# Pass concatenated RGB image through VGG16
x_image = vggnet_model(x_image)

# Flatten output and add dense layers
x_image = Flatten()(x_image)
x_image = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x_image)

# Output layer for classification
model_output = Dense(len(emotion_labels), activation='softmax')(x_image)

# Combine inputs and outputs into a model
model_combined = Model(inputs=model_image_input, outputs=model_output)



# 모델 컴파일
# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)

# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)


import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')


plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []


y_test = []

for image, label in test_dataset:
    test_images.append(image)
    y_test.extend(label.numpy())

y_test_pred = model_combined.predict([np.vstack(test_images)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_images)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)

#VGG19

In [ ]:

# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Lambda

# Model name
model_name = 'vggnet19'

# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}



# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)


    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    return image, label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)


# 모델 정의
image_shape = (48, 48, 1)  # 이미지 데이터의 형태

model_image_input = Input(shape=image_shape)  # 이미지 입력


# 입력 레이어
# 랜드마크와 유클리드 거리 서브넷

# 이미지 서브넷
x_image = Lambda(lambda x: tf.concat([x, x, x], axis=-1))(model_image_input)

# Load VGG16 model pre-trained on ImageNet, excluding top layers
vggnet_model = VGG19(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Freeze VGG16 layers
for layer in vggnet_model.layers:
    layer.trainable = False

# Pass concatenated RGB image through VGG16
x_image = vggnet_model(x_image)

# Flatten output and add dense layers
x_image = Flatten()(x_image)
x_image = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x_image)

# Output layer for classification
model_output = Dense(len(emotion_labels), activation='softmax')(x_image)

# Combine inputs and outputs into a model
model_combined = Model(inputs=model_image_input, outputs=model_output)


# 모델 컴파일
# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)

# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)


import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')


plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []


y_test = []

for image, label in test_dataset:
    test_images.append(image)
    y_test.extend(label.numpy())

y_test_pred = model_combined.predict([np.vstack(test_images)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_images)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)


# EfficientNetB7

In [ ]:

# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.regularizers import l2
from itertools import cycle
# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}

model_name = 'effnetB7'

# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)


    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    return image, label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)


# 모델 정의
image_shape = (48, 48, 1)  # 이미지 데이터의 형태



model_image_input = Input(shape=image_shape)  # 이미지 입력


# 입력 레이어
# 랜드마크와 유클리드 거리 서브넷

# 이미지 서브넷
effnet_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# 이미지 입력
model_image_input = Input(shape=(48, 48, 1))

# 이미지 입력을 EfficientNetB0의 입력 형태에 맞게 조정
x_image = Conv2D(3, (1, 1), kernel_regularizer=l2(0.01))(model_image_input)  # 정규화 추가
x_image = effnet_model(x_image)

x_image = Flatten()(x_image)
x_image = Dropout(0.5)(x_image)  # 드롭아웃 추가
x_image = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x_image)  # 정규화 추가


# 출력 레이어
model_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(x_image)

# 모델 정의
model_combined = Model(inputs= model_image_input, outputs=model_output)

# 모델 컴파일
# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)

# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)

import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')

plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []


y_test = []

for image, label in test_dataset:
    test_images.append(image)
    y_test.extend(label.numpy())

y_test_pred = model_combined.predict([np.vstack(test_images)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_images)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)


# 이미지만 resnet50

In [ ]:

# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.regularizers import l2
# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}

model_name = 'resnet50'


# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)


    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    return image, label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)


# 모델 정의
image_shape = (48, 48, 1)  # 이미지 데이터의 형태

model_image_input = Input(shape=image_shape)  # 이미지 입력


# 입력 레이어
# 랜드마크와 유클리드 거리 서브넷

# 이미지 서브넷
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

model_image_input = Input(shape=(48, 48, 1))
x_image = Conv2D(3, (1, 1))(model_image_input)  # 1x1 convolution to change the number of channels
x_image = resnet_model(x_image)

x_image = Flatten()(x_image)
x_image = Dense(64, activation='relu')(x_image)

model_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(x_image)

model_combined = Model(inputs=model_image_input, outputs=model_output)


# 모델 컴파일
# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)

# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)


import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')


plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []


y_test = []

for image, label in test_dataset:
    test_images.append(image)
    y_test.extend(label.numpy())

y_test_pred = model_combined.predict([np.vstack(test_images)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_images)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)


# 이미지만 EfficientNetB0

In [ ]:

# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.regularizers import l2
from itertools import cycle
# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}
model_name = 'effnetB0'

# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)


    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    return image, label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)


# 모델 정의
image_shape = (48, 48, 1)  # 이미지 데이터의 형태



model_image_input = Input(shape=image_shape)  # 이미지 입력


# 입력 레이어
# 랜드마크와 유클리드 거리 서브넷

# 이미지 서브넷
effnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# 이미지 입력
model_image_input = Input(shape=(48, 48, 1))

# 이미지 입력을 EfficientNetB0의 입력 형태에 맞게 조정
x_image = Conv2D(3, (1, 1), kernel_regularizer=l2(0.01))(model_image_input)  # 정규화 추가
x_image = effnet_model(x_image)

x_image = Flatten()(x_image)
x_image = Dropout(0.5)(x_image)  # 드롭아웃 추가
x_image = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x_image)  # 정규화 추가


# 출력 레이어
model_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(x_image)

# 모델 정의
model_combined = Model(inputs= model_image_input, outputs=model_output)

# 모델 컴파일
# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)

# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)


import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')


plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []


y_test = []

for image, label in test_dataset:
    test_images.append(image)
    y_test.extend(label.numpy())

y_test_pred = model_combined.predict([np.vstack(test_images)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_images)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)


# 이미지만 MobileNet


In [ ]:

# 필요한 라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools
from keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.regularizers import l2
# 감정 레이블 정의
emotion_labels = {
    "happy": 0, "sad": 1, "anger": 2, "surprise": 3,
    "disgust": 4, "neutral": 5, "fear": 6,
}

model_name = 'monet'

# TFRecord 파싱 함수
def parse_and_prepare_tfrecord(tfrecord):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64),
    }
    parsed_features = tf.io.parse_single_example(tfrecord, features)

    image = tf.io.decode_jpeg(parsed_features["image"])
    image = tf.image.resize(image, [48, 48])
    image = tf.image.rgb_to_grayscale(image)
    image = tf.expand_dims(image, -1)
    image = tf.squeeze(image, axis=-1)


    label = tf.one_hot(parsed_features["label"], depth=len(emotion_labels))

    return image, label

# 데이터셋 로드 및 준비
def load_and_prepare_dataset(tfrecord_file, batch_size=None):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_and_prepare_tfrecord)
    if batch_size is not None:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


train_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_train.tfrecord"
val_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_val.tfrecord"
test_tfrecord_file = "/content/drive/MyDrive/clean_0317/Dataset_clean/clean_YES_Z_test.tfrecord"


train_dataset = load_and_prepare_dataset(train_tfrecord_file).batch(batch_size)
val_dataset = load_and_prepare_dataset(val_tfrecord_file).batch(batch_size)
test_dataset = load_and_prepare_dataset(test_tfrecord_file).batch(batch_size)


# 모델 정의
image_shape = (48, 48, 1)  # 이미지 데이터의 형태

model_image_input = Input(shape=image_shape)  # 이미지 입력


# 입력 레이어
# 랜드마크와 유클리드 거리 서브넷

# 이미지 서브넷
mobilenet_model = MobileNet(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

model_image_input = Input(shape=(48, 48, 1))
x_image = Conv2D(3, (1, 1))(model_image_input)  # 1x1 convolution to change the number of channels
x_image = mobilenet_model(x_image)

x_image = Flatten()(x_image)
x_image = BatchNormalization()(x_image)  # 배치 정규화 레이어 추가
x_image = Dense(64, activation='relu')(x_image)
x_image = Dropout(0.5)(x_image)  # 드롭아웃 레이어 추가

model_output = Dense(len(emotion_labels), activation='softmax', kernel_regularizer=l2(0.01))(x_image)

model_combined = Model(inputs=model_image_input, outputs=model_output)


# 모델 컴파일
# 모델 컴파일
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # y_true: 실제 레이블, y_pred: 모델 예측
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        pt = tf.keras.backend.clip(pt, tf.keras.backend.epsilon(), 1.0)  # epsilon 사용
        return -tf.keras.backend.sum(alpha * tf.keras.backend.pow(1 - pt, gamma) * tf.keras.backend.log(pt), axis=-1)

    return focal_loss_fixed
# 모델 컴파일
model_combined.compile(
    optimizer=Adam(learning_rate=learning_rate_combined),  # decreased learning rate
    loss=focal_loss(),
    metrics=['accuracy'])

# 모델 요약
model_combined.summary()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=patience_es, restore_best_weights=True)
train_dataset = train_dataset.shuffle(buffer_size=10000)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=factor_lr, patience=patience_lr, min_lr=1e-6)

# 모델 훈련
history = model_combined.fit(
    x=train_dataset,  # 데이터 증강이 적용된 훈련 데이터셋
    epochs=epochs,
    validation_data=val_dataset,  # 데이터 증강이 적용되지 않은 검증 데이터셋
    callbacks=[early_stopping, reduce_lr]
)

import os

# Google Drive 경로 설정
drive_path = '/content/drive/MyDrive/FINAL'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# 파일 경로 설정
accuracy_loss_path = os.path.join(drive_path, f'{model_name}_accuracy_loss.png')
confusion_matrix_path = os.path.join(drive_path, f'{model_name}_confusion_matrix.png')
emotion_accuracy_path = os.path.join(drive_path, f'{model_name}_emotion_accuracy.png')
precision_recall_curve_path = os.path.join(drive_path, f'{model_name}_precision_recall_curve.png')
roc_curve_path = os.path.join(drive_path, f'{model_name}_roc_curve.png')
pr_curve_data_path = os.path.join(drive_path, f'{model_name}_pr_curve_data.csv')
roc_curve_data_path = os.path.join(drive_path, f'{model_name}_roc_curve_data.csv')
model_save_path_h5 = os.path.join(drive_path, f'{model_name}_model.h5')
model_save_path_keras = os.path.join(drive_path, f'{model_name}_model.keras')
summary_path = os.path.join(drive_path, f'{model_name}_data.txt')

plt.figure(figsize=(14, 5))

# 첫 번째 서브플롯 - 정확도
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title(f' Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# 두 번째 서브플롯 - 손실
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.savefig(accuracy_loss_path)

# Evaluate the model
test_loss, test_accuracy = model_combined.evaluate(test_dataset, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# 혼돈 행렬 생성 및 시각화
test_images = []


y_test = []

for image, label in test_dataset:
    test_images.append(image)
    y_test.extend(label.numpy())

y_test_pred = model_combined.predict([np.vstack(test_images)])

# 모든 이미지와 랜드마크에 대한 예측 수행
y_test = np.argmax(y_test, axis=1)
y_test_pred = np.argmax(y_test_pred, axis=1)

# 혼돈 행렬 생성 및 시각화
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10, 7))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=list(emotion_labels.keys()), yticklabels=list(emotion_labels.keys()))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix')
plt.savefig(confusion_matrix_path)

# 각각의 감정에 대한 분류 정확도 계산 및 시각화
emotion_accuracy_dict = {}
for emotion in emotion_labels:
    emotion_index = emotion_labels[emotion]
    emotion_accuracy = accuracy_score(y_test[y_test == emotion_index], y_test_pred[y_test == emotion_index])
    emotion_accuracy_dict[emotion] = emotion_accuracy
    print(f"Accuracy for {emotion}: {emotion_accuracy:.2f}")

# 감정별 정확도를 바 차트로 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x=list(emotion_accuracy_dict.keys()), y=list(emotion_accuracy_dict.values()))
plt.title(f'Accuracy for Each Emotion')
plt.savefig(emotion_accuracy_path)

# 정밀도, 재현율, F1 스코어 계산
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_recall_curve

# 정밀도, 재현율, F1 스코어 계산
precision = precision_score(y_test, y_test_pred, average='macro')
recall = recall_score(y_test, y_test_pred, average='macro')
f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

# AUC 스코어 계산
# 감정 레이블이 다중 클래스이므로, AUC 계산을 위해 one-vs-rest 방식을 적용해야 합니다.
# 각 클래스에 대해 확률을 계산해야 하므로, 모델의 예측을 확률로 변경합니다.
# Model prediction
y_test_prob = model_combined.predict([np.vstack(test_images)])
roc_auc = roc_auc_score(y_test, y_test_prob, multi_class='ovr', average="macro")
print(f"{model_name} ROC AUC Score: {roc_auc:.2f}")

# Plot PR curve
precision_dict = {}
recall_dict = {}
average_precision_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    precision_dict[emotion], recall_dict[emotion], _ = precision_recall_curve(y_test == i, y_test_prob[:, i])
    average_precision_dict[emotion] = average_precision_score(y_test == i, y_test_prob[:, i])
    plt.plot(recall_dict[emotion], precision_dict[emotion], lw=2,
             label=f'{emotion} (AP={average_precision_dict[emotion]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall curve')
plt.legend(loc="best")
plt.savefig(precision_recall_curve_path)

# Plot ROC curve
fpr_dict = {}
tpr_dict = {}
plt.figure(figsize=(10, 6))

for i, emotion in enumerate(emotion_labels):
    fpr_dict[emotion], tpr_dict[emotion], _ = roc_curve(y_test == i, y_test_prob[:, i])
    plt.plot(fpr_dict[emotion], tpr_dict[emotion], lw=2, label=f'{emotion} (area = {auc(fpr_dict[emotion], tpr_dict[emotion]):.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title(f'ROC curve')
plt.legend(loc="best")
plt.savefig(roc_curve_path)

# Create empty dataframes to store PR and ROC curve data
pr_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AP', 'Recall', 'Precision'])
roc_curve_data = pd.DataFrame(columns=['Model', 'Emotion', 'AUC', 'FPR', 'TPR'])

# Store PR curve data
temp_data = []

for i, emotion in enumerate(emotion_labels):
    for r, p in zip(recall_dict[emotion], precision_dict[emotion]):
        temp_data.append({'Model': model_name, 'Emotion': emotion, 'AP': average_precision_dict[emotion], 'Recall': r, 'Precision': p})

pr_curve_data = pd.DataFrame(temp_data)

# Store ROC curve data
temp_roc_data = []

for i, emotion in enumerate(emotion_labels):
    for f, t in zip(fpr_dict[emotion], tpr_dict[emotion]):
        temp_roc_data.append({
            'Model': model_name,
            'Emotion': emotion,
            'AUC': auc(fpr_dict[emotion], tpr_dict[emotion]),
            'FPR': f,
            'TPR': t
        })

roc_curve_data = pd.DataFrame(temp_roc_data)
pr_curve_data.to_csv(pr_curve_data_path, index=False)
roc_curve_data.to_csv(roc_curve_data_path, index=False)

with open(summary_path, 'w') as f:
    f.write(f"Test Loss: {test_loss}\n")
    f.write(f"Test Accuracy: {test_accuracy}\n")
    for emotion, accuracy in emotion_accuracy_dict.items():
        f.write(f"Accuracy for {emotion}: {accuracy:.3f}\n")
    f.write(f"Precision: {precision:.3f}\n")
    f.write(f"Recall: {recall:.3f}\n")
    f.write(f"F1 Score: {f1:.3f}\n")
    f.write(f"{model_name} ROC AUC Score: {roc_auc:.3f}\n")

model_combined.save(model_save_path_h5)
model_combined.save(model_save_path_keras)


# PR

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 파일 목록
pr_files = [
    '/content/drive/MyDrive/FINAL/1-d_pr_curve_data.csv', '/content/drive/MyDrive/FINAL/basic 3 input_pr_curve_data.csv',
    '/content/drive/MyDrive/FINAL/freezing_pr_curve_data.csv', '/content/drive/MyDrive/FINAL/GAN_pr_curve_data.csv',
    '/content/drive/MyDrive/FINAL/monet_pr_curve_data.csv', '/content/drive/MyDrive/FINAL/resnet50_pr_curve_data.csv',
    '/content/drive/MyDrive/FINAL/vggnet16_pr_curve_data.csv', '/content/drive/MyDrive/FINAL/vggnet19_pr_curve_data.csv',
    '/content/drive/MyDrive/FINAL/weighted_pr_curve_data.csv', '/content/drive/MyDrive/FINAL/effnetB0_pr_curve_data.csv'
]

# Print file paths for verification
print("Checking file paths:")
for file in pr_files:
    print(file)

# 감정 목록
emotions = ["happy", "sad", "anger", "surprise", "disgust", "neutral", "fear"]

# 각 감정별 평균 AP 저장
average_precision_dict = {}

# 각 감정별로 PR 곡선 그리기
for emotion in emotions:
    plt.figure(figsize=(10, 8))

    # 각 파일에 대해 평균 precision-recall 곡선을 계산
    model_data = []
    for file in pr_files:
        df = pd.read_csv(file)
        model_name = file.split('/')[-1].split('_pr_curve_data.csv')[0]  # 파일 이름에서 모델 이름 추출
        # 해당 감정의 데이터만 추출
        emotion_data = df[df['Emotion'] == emotion]

        # Precision-Recall 곡선 그리기
        precision = emotion_data['Precision']
        recall = emotion_data['Recall']
        average_precision = np.mean(emotion_data['AP'])
        average_precision_dict[emotion] = average_precision
        model_data.append((model_name, recall, precision, average_precision))

    # AP 높은 순서대로 정렬
    model_data.sort(key=lambda x: x[3], reverse=True)

    for model_name, recall, precision, average_precision in model_data:
        plt.plot(recall, precision, label=f'{model_name} (AP={average_precision:.3f})')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve for {emotion} Emotion')
    plt.legend()
    plt.grid(True)
    plt.show()


# ROC

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 파일 목록
roc_files = [
    '/content/drive/MyDrive/FINAL/1-d_roc_curve_data.csv', '/content/drive/MyDrive/FINAL/basic 3 input_roc_curve_data.csv',
    '/content/drive/MyDrive/FINAL/freezing_roc_curve_data.csv','/content/drive/MyDrive/FINAL/GAN_roc_curve_data.csv',
    '/content/drive/MyDrive/FINAL/monet_roc_curve_data.csv', '/content/drive/MyDrive/FINAL/resnet50_roc_curve_data.csv',
    '/content/drive/MyDrive/FINAL/vggnet16_roc_curve_data.csv', '/content/drive/MyDrive/FINAL/vggnet19_roc_curve_data.csv',
    '/content/drive/MyDrive/FINAL/weighted_roc_curve_data.csv', '/content/drive/MyDrive/FINAL/effnetB0_roc_curve_data.csv'
]

# 감정 목록
emotions = ["happy", "sad", "anger", "surprise", "disgust", "neutral", "fear"]

# ROC 커브 데이터 저장
roc_data = {}

for emotion in emotions:
    plt.figure(figsize=(10, 8))

    model_data = []
    for file in roc_files:
        df = pd.read_csv(file)
        model_name = file.split('/')[-1].split('_roc_curve_data.csv')[0]  # Extract model name from file path
        emotion_data = df[df['Emotion'] == emotion]


        # Check if 'FPR' exists in emotion_data
        if 'FPR' in emotion_data.columns:
            fpr = emotion_data['FPR']
            tpr = emotion_data['TPR']
            auc = np.mean(emotion_data['AUC'])
            model_data.append((model_name, fpr, tpr, auc))
        else:
            print(f"Column 'FPR' not found in {file}")

    model_data.sort(key=lambda x: x[3], reverse=True)

    for model_name, fpr, tpr, auc in model_data:
        plt.plot(fpr, tpr, label=f'{model_name} (AUC={auc:.3f})')

    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('True Positive Rate (TPR)')
    plt.title(f'ROC Curve for {emotion} Emotion')
    plt.legend()
    plt.grid(True)
    plt.show()



# 결과 정리

In [ ]:
import pandas as pd

# 파일 경로 리스트
file_paths = [
    '/content/drive/MyDrive/FINAL/Image Only_data.txt',
    '/content/drive/MyDrive/FINAL/Landmark Only_data.txt',
    '/content/drive/MyDrive/FINAL/ZValue Only_data.txt',
    '/content/drive/MyDrive/FINAL/Image & Landmark_data.txt',
    '/content/drive/MyDrive/FINAL/Image & ZValue_data.txt',
    '/content/drive/MyDrive/FINAL/Landmark & ZValue_data.txt',
    '/content/drive/MyDrive/FINAL/Image & Landmark & ZValue_data.txt',
    '/content/drive/MyDrive/FINAL/Image & ZValue & Landmark_data.txt',
    '/content/drive/MyDrive/FINAL/ZValue & Landmark & Image_data.txt',
    '/content/drive/MyDrive/FINAL/1-d_data.txt',
    '/content/drive/MyDrive/FINAL/GAN_data.txt',
    '/content/drive/MyDrive/FINAL/basic 3 input_data.txt',
    '/content/drive/MyDrive/FINAL/freezing_data.txt',
    '/content/drive/MyDrive/FINAL/weighted_data.txt',
    '/content/drive/MyDrive/FINAL/effnetB0_data.txt',
    '/content/drive/MyDrive/FINAL/effnetB7_data.txt',
    '/content/drive/MyDrive/FINAL/monet_data.txt',
    '/content/drive/MyDrive/FINAL/resnet50_data.txt',
    '/content/drive/MyDrive/FINAL/vggnet16_data.txt',
    '/content/drive/MyDrive/FINAL/vggnet19_data.txt',

]

# 결과를 저장할 데이터프레임 생성
result_data = pd.DataFrame(columns=[
    'Model', 'Test Loss', 'Test Accuracy', 'Accuracy for Happy', 'Accuracy for Sad',
    'Accuracy for Anger', 'Accuracy for Surprise', 'Accuracy for Disgust',
    'Accuracy for Neutral', 'Accuracy for Fear', 'Precision', 'Recall', 'F1 Score', 'ROC AUC Score'
])

# 파일을 읽어서 데이터프레임에 추가하는 함수 정의
def process_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()

            # 모델 이름 추출
            model_name = file_path.split('/')[-1].split('_data.txt')[0]  # 파일 경로에서 모델 이름 추출

            # 성능 지표 추출
            test_loss = float(lines[0].split(': ')[1].strip())
            test_accuracy = float(lines[1].split(': ')[1].strip())
            accuracy_happy = float(lines[2].split(': ')[1].strip())
            accuracy_sad = float(lines[3].split(': ')[1].strip())
            accuracy_anger = float(lines[4].split(': ')[1].strip())
            accuracy_surprise = float(lines[5].split(': ')[1].strip())
            accuracy_disgust = float(lines[6].split(': ')[1].strip())
            accuracy_neutral = float(lines[7].split(': ')[1].strip())
            accuracy_fear = float(lines[8].split(': ')[1].strip())
            precision = float(lines[9].split(': ')[1].strip())
            recall = float(lines[10].split(': ')[1].strip())
            f1_score = float(lines[11].split(': ')[1].strip())
            roc_auc_score = float(lines[12].split(': ')[1].strip())

            # 데이터프레임에 추가
            result_data.loc[len(result_data)] = [
                model_name, test_loss, test_accuracy, accuracy_happy, accuracy_sad,
                accuracy_anger, accuracy_surprise, accuracy_disgust, accuracy_neutral,
                accuracy_fear, precision, recall, f1_score, roc_auc_score
            ]

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error reading file: {e}")

# 각 파일을 처리하여 데이터프레임에 추가
for file_path in file_paths:
    process_file(file_path)

# 결과를 CSV 파일로 저장
result_csv_path = '/content/drive/MyDrive/FINAL/model_performance_comparison.csv'
result_data.to_csv(result_csv_path, index=False)

# 저장된 CSV 파일 읽기
read_result_data = pd.read_csv(result_csv_path)
print(read_result_data)
